In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [337]:
import warnings
warnings.filterwarnings('ignore')
def setup_seed(seed=0):
    import torch
    import os
    import numpy as np
    import random
    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    if torch.cuda.is_available():
        # torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)
def sliding_windows(data,prediction, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = prediction[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)
from sklearn.metrics import mean_squared_error
def process_data(data,periods,year):
  data['clean_review'] = data.text.apply(clean_text)
  data
  if year==30:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_US.csv',index_col=[0])
  elif year==10:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_10_US.csv',index_col=[0])
  elif year==5:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_5_US.csv',index_col=[0])
  elif year==0.3:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_3m_US.csv',index_col=[0])
  interest_rate=interest_rate.sort_values(by='Date')
  interest_rate['range']=list(range(len(interest_rate)))
  data['change']=np.nan
  for i in range(len(data)):
    try:
      index=int(interest_rate[interest_rate['Date']<=int(data.iloc[i,0])]['range'].tail(1))
      data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
    except:
      pass
  new_raw=data.dropna()
  
  return new_raw
def split_data(data,ratio,features,seq_length):
  vectorizer_tfidf=TfidfVectorizer(max_features=features)
  total_vsm_tfidf=vectorizer_tfidf.fit_transform(data.clean_review).toarray()
  seq_length = 4
  x, y = sliding_windows(total_vsm_tfidf,list(data.change), seq_length)
  train_size = int(len(x) *ratio)
  test_size = len(x) - train_size
  train_Y=np.array(x[0:train_size])
  from sklearn.preprocessing import RobustScaler
  train_Y=np.array(y[0:train_size])
  setup_seed(1)
  scaler=RobustScaler().fit(train_Y.reshape(-1, 1) )
  train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
  train_Y = Variable(torch.Tensor(train_Y))
  train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
  test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
  test_Y = np.array(y[train_size:len(y)])
  return train_X,train_Y,test_X,test_Y
def taining(train_X,train_Y,test_X,test_Y,data,ratio,seq_length,features,l2,seed):
  #train_X,train_Y,test_X,test_Y=split_data(data,ratio,features,seq_length)
  num_epochs = 400
  learning_rate = 0.01
  input_size = features
  hidden_size = 2
  num_layers = 1
  num_classes = 1
  setup_seed(seed)
  lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=l2)
  from sklearn.metrics import mean_squared_error
  for epoch in range(num_epochs):
      outputs = lstm(train_X)
      optimizer.zero_grad()
      loss = criterion(outputs, train_Y[0])
      loss.backward()
      optimizer.step()
      predTrain = lstm(test_X)
      print(predTrain.shape)
      tmp=pd.DataFrame(list(test_Y))
      tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
      r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
      print('IC',round(tmp.corr().iloc[0,1],5),
            'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
            'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
            'loss',round(r,5))
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [325]:
import os
path = "/content/gdrive/MyDrive/JPM/centralbank_analysis/data/FOMC/statement/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [326]:
file_list_time=[file_list[i].split('-')[0][-4:]+file_list[i].split('-')[1]+file_list[i].split('-')[2][:2] for i in range(len(file_list))]

In [327]:
repeat_sentence='The Federal Reserve, the central bank of the United States, provides the nation with a safe, flexible, and stable monetary and financial system. Monetary Policy Principles and Practice Review of Monetary Policy Strategy, Tools, and Communications Banking Applications & Legal Developments Reserve Bank Payment Services & Data Financial Market Utilities & Infrastructures Exchange Rates and International Data Micro Data Reference Manual (MDRM) Money Stock and Reserve Balances' 

In [328]:
statement_text=dict()
for i in range(len(file_list)):
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        if ('Voting for the FOMC monetary policy action were' in line):
          break
        contents.append(line.strip())
  text = ' '.join(contents)
  text=text.replace('Â\xa0',' ')
  text=text.replace(repeat_sentence,"")
  if len(text.split(' '))>100:
    statement_text[file_list_time[i]]='statement '+text
  else:
    print(i,len(text.split(' ')),text)

3 99 Chairman Alan Greenspan announced today that the Federal Open Market Committee decided to increase slightly the degree of pressure on reserve positions. The action is expected to be associated with a small increase in short-term money market interest rates. The decision was taken to move toward a less accommodative stance in monetary policy in order to sustain and enhance the economic expansion. Chairman Greenspan decided to announce this action immediately so as to avoid any misunderstanding of the Committee's purposes, given the fact that this is the first firming of reserve market conditions by the Committee since early 1989.
15 36 Chairman Alan Greenspan announced today that the Federal Reserve will increase slightly the degree of pressure on reserve positions. This action is expected to be associated with a small increase in short-term money market interest rates.
41 71 Chairman Alan Greenspan announced today that the Federal Open Market Committee decided to decrease slightly

In [329]:
path = "/content/gdrive/MyDrive/JPM/centralbank_analysis/data/FOMC/speech/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [330]:
file_list_time_speech=[file_list[i].split('-')[0][-4:]+file_list[i].split('-')[1]+file_list[i].split('-')[2][:2] for i in range(len(file_list))]

In [331]:
speech_dicter={}
path_save = "/content/gdrive/MyDrive/JPM/centralbank_analysis/data/FOMC/text_summarization/speech/"
for i in range(len(file_list)):
  #print(i,file_list[i])
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    #print(line)
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        contents.append(line.strip().replace('Â',' '))
  
  text = ' '.join(contents)
  text=text.replace('Â\xa0',' ')
  text=text.replace(repeat_sentence,"")
  #print(text)
  if (len(text.split(' '))<100):
    print('!',i,text)
  else:
    speech_dicter[file_list_time_speech[i]]='speech '+text

! 84 Reflections on the Capital Goods Overhang
! 104 
! 110 Corporate governance, the subject of our conference, has evolved over the past century to more effectively promote the allocation of the nation s savings to its most productive uses.  And, generally speaking, the resulting structure of business incentives, reporting, and accountability has served us well.  We could not have achieved our current level of national productivity if corporate governance had been deeply flawed.
! 154 
! 159 
! 169 
! 188 
! 205 
! 254 
! 321 Asset Prices and Monetary Policy
! 333 Thank you, Tom.  On behalf of the Federal Reserve, I am pleased to be with you all as we unveil the newly designed $20 note.
! 370 I am pleased to appear once again before the Independent Community Bankers of America.  There can be little doubt that the business environment in which you have operated over the past year has been challenging.
! 406 
! 469 
! 519 Thank you. I am pleased to have the opportunity to help launch 

In [332]:
import pandas as pd
raw_data_speech=pd.DataFrame({'date':speech_dicter.keys(),'text':speech_dicter.values(),'type':['speech' for i in range(len(speech_dicter.values()))]})
raw_data_speech.sort_values(by='date',inplace=True)
raw_data_speech

,date,text,type
1083,19960613,speech Remarks by Chairman Alan Greenspan Bank...,speech
488,19960618,"speech Remarks by Governor Edward W. Kelley, J...",speech
15,19960908,speech Monetary Policy Objectives and Strategy...,speech
1232,19960919,speech Remarks by Chairman Alan Greenspan Regu...,speech
1236,19961002,speech Remarks by Governor Lawrence B. Lindsey...,speech
...,...,...,...
810,20220221,speech At the American Bankers Association Co...,speech
700,20220224,"speech At the Economic Forecast Project, Univ...",speech
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech


In [333]:
raw_data_statement=pd.DataFrame({'date':statement_text.keys(),'text':statement_text.values(),'type':['statement' for i in range(len(statement_text.values()))]})
raw_data_statement.sort_values(by='date',inplace=True)
raw_data_statement

,date,text,type
186,19940517,statement The Federal Reserve today announced ...,statement
146,19940816,statement The Federal Reserve announced today ...,statement
177,19941115,statement The Federal Reserve Board today appr...,statement
153,19950201,statement The Federal Reserve Board today appr...,statement
25,19960131,statement The Federal Reserve today announced ...,statement
...,...,...,...
149,20210922,statement For release at 2:00 p.m. EDT The Fe...,statement
88,20211103,statement For release at 2:00 p.m. EDT The Fe...,statement
33,20211215,statement For release at 2:00 p.m. EST The Fe...,statement
66,20220126,statement For release at 2:00 p.m. EST Indica...,statement


In [126]:
raw_data=raw_data_statement#pd.concat([raw_data_speech,raw_data_statement],axis=0)
raw_data.sort_values(by='date',inplace=True)

In [127]:
raw_data

,date,text,type
186,19940517,statement The Federal Reserve today announced ...,statement
146,19940816,statement The Federal Reserve announced today ...,statement
177,19941115,statement The Federal Reserve Board today appr...,statement
153,19950201,statement The Federal Reserve Board today appr...,statement
25,19960131,statement The Federal Reserve today announced ...,statement
...,...,...,...
149,20210922,statement For release at 2:00 p.m. EDT The Fe...,statement
88,20211103,statement For release at 2:00 p.m. EDT The Fe...,statement
33,20211215,statement For release at 2:00 p.m. EST The Fe...,statement
66,20220126,statement For release at 2:00 p.m. EST Indica...,statement


In [128]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
raw_data=pd.concat([raw_data_statement,raw_data_speech],axis=0)
raw_data.sort_values(by='date',inplace=True)
eng_stopwords = stopwords.words('english')
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)

raw_data['clean_review'] = raw_data.text.apply(clean_text)
raw_data

,date,text,type,clean_review
186,19940517,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced two ...
146,19940816,statement The Federal Reserve announced today ...,statement,statement federal reserve announced today foll...
177,19941115,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...
153,19950201,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...
25,19960131,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced foll...
...,...,...,...,...
700,20220224,"speech At the Economic Forecast Project, Univ...",speech,speech economic forecast project university ca...
188,20220316,statement For release at 2:00 p.m. EDT Indica...,statement,statement release p edt indicators economic ac...
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech,speech fed listens helping youth thrive discus...
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech,speech chair pro tempore jerome h powell polic...


In [303]:
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_US.csv',index_col=[0])
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))
raw_data['change']=np.nan
for i in range(len(raw_data)):
  try:
    index=int(interest_rate[interest_rate['Date']<=int(raw_data.iloc[i,0])]['range'].tail(1))
    raw_data.iloc[i,-1]=(interest_rate.iloc[index+10,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
  except:
    pass
new_raw=raw_data.dropna()
new_raw

,date,text,type,clean_review,change
186,19940517,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced two ...,0.013896
146,19940816,statement The Federal Reserve announced today ...,statement,statement federal reserve announced today foll...,0.010990
177,19941115,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...,-0.005525
153,19950201,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...,-0.021935
25,19960131,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced foll...,0.009535
...,...,...,...,...,...
700,20220224,"speech At the Economic Forecast Project, Univ...",speech,speech economic forecast project university ca...,-0.019494
188,20220316,statement For release at 2:00 p.m. EDT Indica...,statement,statement release p edt indicators economic ac...,0.025123
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech,speech fed listens helping youth thrive discus...,0.017246
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech,speech chair pro tempore jerome h powell polic...,-0.024148


In [276]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [277]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [278]:
features=150
vectorizer_freq = CountVectorizer(max_features = features) 
total_vsm_freq = vectorizer_freq.fit_transform(new_raw.clean_review).toarray()
print("以词频为元素的文本-单词矩阵的维度是：\n\n",total_vsm_freq.shape)
from scipy import stats
vectorizer_tfidf=TfidfVectorizer(max_features=features)
total_vsm_tfidf=vectorizer_tfidf.fit_transform(new_raw.clean_review).toarray()


以词频为元素的文本-单词矩阵的维度是：

 (1512, 150)


In [279]:
new_raw.dropna()

,date,text,type,clean_review,change
186,19940517,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced two ...,0.013896
146,19940816,statement The Federal Reserve announced today ...,statement,statement federal reserve announced today foll...,0.010990
177,19941115,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...,-0.005525
153,19950201,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...,-0.021935
25,19960131,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced foll...,0.009535
...,...,...,...,...,...
700,20220224,"speech At the Economic Forecast Project, Univ...",speech,speech economic forecast project university ca...,-0.019494
188,20220316,statement For release at 2:00 p.m. EDT Indica...,statement,statement release p edt indicators economic ac...,0.025123
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech,speech fed listens helping youth thrive discus...,0.017246
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech,speech chair pro tempore jerome h powell polic...,-0.024148


In [291]:
def sliding_windows(data,prediction, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = prediction[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)
seq_length = 5
x, y = sliding_windows(total_vsm_tfidf,list(new_raw.change), seq_length)
train_size = int(len(x) * 0.7)
test_size = len(x) - train_size
train_Y=np.array(x[0:train_size])
from sklearn.preprocessing import RobustScaler
train_Y=np.array(y[0:train_size])
scaler =RobustScaler().fit(train_Y.reshape(-1, 1) )
train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
train_Y=Variable(torch.Tensor(train_Y))
train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
test_Y = np.array(y[train_size:len(y)])

In [344]:
features=150
seq_length = 5
ratio_=0.7
periods=10
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_10_US.csv',index_col=[0])
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))
raw_data['change']=np.nan
for i in range(len(raw_data)):
  try:
    index=int(interest_rate[interest_rate['Date']<=int(raw_data.iloc[i,0])]['range'].tail(1))
    raw_data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
  except:
    pass
new_raw=raw_data.dropna()
vectorizer_freq = CountVectorizer(max_features = features) 
total_vsm_freq = vectorizer_freq.fit_transform(new_raw.clean_review).toarray()
print("以词频为元素的文本-单词矩阵的维度是：\n\n",total_vsm_freq.shape)
from scipy import stats
vectorizer_tfidf=TfidfVectorizer(max_features=features)
total_vsm_tfidf=vectorizer_tfidf.fit_transform(new_raw.clean_review).toarray()
x, y = sliding_windows(total_vsm_tfidf,list(new_raw.change), seq_length)
train_size = int(len(x) * ratio_)
test_size = len(x) - train_size
train_Y=np.array(x[0:train_size])
from sklearn.preprocessing import RobustScaler
train_Y=np.array(y[0:train_size])
scaler =RobustScaler().fit(train_Y.reshape(-1, 1) )
train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
train_Y=Variable(torch.Tensor(train_Y))
train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
test_Y = np.array(y[train_size:len(y)])

以词频为元素的文本-单词矩阵的维度是：

 (1512, 150)


In [347]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.7,5,150,0.001,3)

torch.Size([452, 1])
IC 0.03261 IC spear -0.00804 P_value 0.86463 loss 0.07573
Epoch: 0, loss: 0.54605
torch.Size([452, 1])
IC 0.03611 IC spear -0.00172 P_value 0.97093 loss 0.0633
Epoch: 1, loss: 0.50734
torch.Size([452, 1])
IC 0.03879 IC spear 0.00161 P_value 0.97284 loss 0.0533
Epoch: 2, loss: 0.47458
torch.Size([452, 1])
IC 0.04068 IC spear 0.00293 P_value 0.95052 loss 0.0452
Epoch: 3, loss: 0.44682
torch.Size([452, 1])
IC 0.04189 IC spear 0.00614 P_value 0.89644 loss 0.03859
Epoch: 4, loss: 0.42311
torch.Size([452, 1])
IC 0.04253 IC spear 0.00927 P_value 0.84412 loss 0.03308
Epoch: 5, loss: 0.40258
torch.Size([452, 1])
IC 0.04278 IC spear 0.01162 P_value 0.80541 loss 0.02839
Epoch: 6, loss: 0.38440
torch.Size([452, 1])
IC 0.04277 IC spear 0.01346 P_value 0.77539 loss 0.02431
Epoch: 7, loss: 0.36790
torch.Size([452, 1])
IC 0.04265 IC spear 0.01472 P_value 0.755 loss 0.02068
Epoch: 8, loss: 0.35250
torch.Size([452, 1])
IC 0.04251 IC spear 0.01685 P_value 0.72097 loss 0.01741
Epoch: 

In [324]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.7,5,150,0.001,5)

torch.Size([452, 1])
IC -0.04734 IC spear -0.00891 P_value 0.85013 loss 0.25374
Epoch: 0, loss: 0.00609
torch.Size([452, 1])
IC -0.05163 IC spear -0.01296 P_value 0.78343 loss 0.22881
Epoch: 1, loss: 0.00298
torch.Size([452, 1])
IC -0.05518 IC spear -0.01513 P_value 0.74838 loss 0.20768
Epoch: 2, loss: 0.00134
torch.Size([452, 1])
IC -0.05756 IC spear -0.01838 P_value 0.69678 loss 0.19256
Epoch: 3, loss: 0.00100
torch.Size([452, 1])
IC -0.05869 IC spear -0.02002 P_value 0.67124 loss 0.18463
Epoch: 4, loss: 0.00142
torch.Size([452, 1])
IC -0.05874 IC spear -0.01931 P_value 0.68222 loss 0.18306
Epoch: 5, loss: 0.00187
torch.Size([452, 1])
IC -0.05798 IC spear -0.0187 P_value 0.6918 loss 0.18619
Epoch: 6, loss: 0.00194
torch.Size([452, 1])
IC -0.05663 IC spear -0.01656 P_value 0.72545 loss 0.19247
Epoch: 7, loss: 0.00165
torch.Size([452, 1])
IC -0.05492 IC spear -0.01426 P_value 0.76236 loss 0.20053
Epoch: 8, loss: 0.00122
torch.Size([452, 1])
IC -0.0531 IC spear -0.01147 P_value 0.80788 

In [281]:
import warnings
warnings.filterwarnings('ignore')
def setup_seed(seed=0):
    import torch
    import os
    import numpy as np
    import random
    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    if torch.cuda.is_available():
        # torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [282]:
import warnings
warnings.filterwarnings('ignore')

In [283]:
def setup_seed(seed=0):
    import torch
    import os
    import numpy as np
    import random
    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    if torch.cuda.is_available():
        # torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子

In [315]:

num_epochs = 400
learning_rate = 0.01

input_size = 150
hidden_size = 2
num_layers = 1

num_classes = 1
setup_seed(2)
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=0.001)
# Train the model
from sklearn.metrics import mean_squared_error
for epoch in range(num_epochs):
    outputs = lstm(train_X)
    optimizer.zero_grad()
    # obtain the loss function
    loss = criterion(outputs, train_Y[0])
    loss.backward()
    optimizer.step()
    predTrain = lstm(test_X)
    print(predTrain.shape)
    tmp=pd.DataFrame(list(test_Y))
    tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
    r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
    print('IC',round(tmp.corr().iloc[0,1],5),
          'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
          'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
          'loss',round(r,5))
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

torch.Size([452, 1])
IC 0.05365 IC spear 0.05411 P_value 0.25095 loss 0.20131
Epoch: 0, loss: 0.00691
torch.Size([452, 1])
IC 0.05422 IC spear 0.05592 P_value 0.23541 loss 0.22755
Epoch: 1, loss: 0.00324
torch.Size([452, 1])
IC 0.05504 IC spear 0.05761 P_value 0.22155 loss 0.2466
Epoch: 2, loss: 0.00226
torch.Size([452, 1])
IC 0.05598 IC spear 0.05916 P_value 0.20933 loss 0.25568
Epoch: 3, loss: 0.00262
torch.Size([452, 1])
IC 0.05698 IC spear 0.05956 P_value 0.20624 loss 0.2557
Epoch: 4, loss: 0.00304
torch.Size([452, 1])
IC 0.05804 IC spear 0.06075 P_value 0.19734 loss 0.24949
Epoch: 5, loss: 0.00294
torch.Size([452, 1])
IC 0.05911 IC spear 0.06268 P_value 0.18346 loss 0.23978
Epoch: 6, loss: 0.00245
torch.Size([452, 1])
IC 0.06021 IC spear 0.06306 P_value 0.18078 loss 0.22886
Epoch: 7, loss: 0.00190
torch.Size([452, 1])
IC 0.06132 IC spear 0.06319 P_value 0.17987 loss 0.21859
Epoch: 8, loss: 0.00152
torch.Size([452, 1])
IC 0.06244 IC spear 0.06247 P_value 0.18493 loss 0.2104
Epoch: 

In [299]:
tmp.corr()

,0,1
0,1.000000,0.036544
1,0.036544,1.000000


In [248]:
\

0.03748830901214511

In [287]:

num_epochs = 300
learning_rate = 0.01

input_size = 150
hidden_size = 2
num_layers = 1

num_classes = 1
setup_seed(3)
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=0.0001)
# Train the model
for epoch in range(num_epochs):
    outputs = lstm(train_X)
    optimizer.zero_grad()
    # obtain the loss function
    loss = criterion(outputs, train_Y)
    loss.backward()
    optimizer.step()
    predTrain = lstm(test_X)
    print(predTrain.shape)
    tmp=pd.DataFrame(list(test_Y))
    tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
    r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
   # prin
    print('IC',round(tmp.corr().iloc[0,1],5),
          'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
          'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
          'loss',round(r,5))
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

torch.Size([452, 1])
IC 0.02087 IC spear -0.01487 P_value 0.75258 loss 0.07569
Epoch: 0, loss: 0.71272
torch.Size([452, 1])
IC 0.02447 IC spear -0.00975 P_value 0.83617 loss 0.0631
Epoch: 1, loss: 0.69908
torch.Size([452, 1])
IC 0.02763 IC spear -0.00619 P_value 0.89554 loss 0.05313
Epoch: 2, loss: 0.68843
torch.Size([452, 1])
IC 0.03042 IC spear -0.00467 P_value 0.92109 loss 0.04519
Epoch: 3, loss: 0.68014
torch.Size([452, 1])
IC 0.03292 IC spear 0.00011 P_value 0.99806 loss 0.03878
Epoch: 4, loss: 0.67364
torch.Size([452, 1])
IC 0.03512 IC spear 0.00375 P_value 0.93657 loss 0.0335
Epoch: 5, loss: 0.66844
torch.Size([452, 1])
IC 0.03704 IC spear 0.00897 P_value 0.84919 loss 0.02906
Epoch: 6, loss: 0.66420
torch.Size([452, 1])
IC 0.03866 IC spear 0.01347 P_value 0.77525 loss 0.02524
Epoch: 7, loss: 0.66067
torch.Size([452, 1])
IC 0.04 IC spear 0.01702 P_value 0.71815 loss 0.02191
Epoch: 8, loss: 0.65766
torch.Size([452, 1])
IC 0.04108 IC spear 0.02082 P_value 0.65882 loss 0.01896
Epoch

In [320]:
from sklearn.metrics import mean_squared_error
def process_data(data,periods,year):
  data['clean_review'] = data.text.apply(clean_text)
  data
  if year==30:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_US.csv',index_col=[0])
  elif year==10:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_10_US.csv',index_col=[0])
  elif year==5:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_5_US.csv',index_col=[0])
  elif year==0.3:
    interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_3m_US.csv',index_col=[0])
  interest_rate=interest_rate.sort_values(by='Date')
  interest_rate['range']=list(range(len(interest_rate)))
  data['change']=np.nan
  for i in range(len(data)):
    try:
      index=int(interest_rate[interest_rate['Date']<=int(data.iloc[i,0])]['range'].tail(1))
      data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
    except:
      pass
  new_raw=data.dropna()
  
  return new_raw
def split_data(data,ratio,features,seq_length):
  vectorizer_tfidf=TfidfVectorizer(max_features=features)
  total_vsm_tfidf=vectorizer_tfidf.fit_transform(data.clean_review).toarray()
  seq_length = 4
  x, y = sliding_windows(total_vsm_tfidf,list(data.change), seq_length)
  train_size = int(len(x) *ratio)
  test_size = len(x) - train_size
  train_Y=np.array(x[0:train_size])
  from sklearn.preprocessing import RobustScaler
  train_Y=np.array(y[0:train_size])
  setup_seed(1)
  scaler=RobustScaler().fit(train_Y.reshape(-1, 1) )
  train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
  train_Y = Variable(torch.Tensor(train_Y))
  train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
  test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
  test_Y = np.array(y[train_size:len(y)])
  return train_X,train_Y,test_X,test_Y
def taining(train_X,train_Y,test_X,test_Y,data,ratio,seq_length,features,l2,seed):
  #train_X,train_Y,test_X,test_Y=split_data(data,ratio,features,seq_length)
  num_epochs = 400
  learning_rate = 0.01
  input_size = features
  hidden_size = 2
  num_layers = 1
  num_classes = 1
  setup_seed(seed)
  lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=l2)
  from sklearn.metrics import mean_squared_error
  for epoch in range(num_epochs):
      outputs = lstm(train_X)
      optimizer.zero_grad()
      loss = criterion(outputs, train_Y[0])
      loss.backward()
      optimizer.step()
      predTrain = lstm(test_X)
      print(predTrain.shape)
      tmp=pd.DataFrame(list(test_Y))
      tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
      r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
      print('IC',round(tmp.corr().iloc[0,1],5),
            'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
            'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
            'loss',round(r,5))
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [316]:
new_raw=process_data(raw_data,10,30)

In [317]:
new_raw

,date,text,type,clean_review,change
186,19940517,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced two ...,0.013896
146,19940816,statement The Federal Reserve announced today ...,statement,statement federal reserve announced today foll...,0.010990
177,19941115,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...,-0.005525
153,19950201,statement The Federal Reserve Board today appr...,statement,statement federal reserve board today approved...,-0.021935
25,19960131,statement The Federal Reserve today announced ...,statement,statement federal reserve today announced foll...,0.009535
...,...,...,...,...,...
700,20220224,"speech At the Economic Forecast Project, Univ...",speech,speech economic forecast project university ca...,-0.019494
188,20220316,statement For release at 2:00 p.m. EDT Indica...,statement,statement release p edt indicators economic ac...,0.025123
737,20220318,speech At Fed Listens: Helping Youth ThriveÃ¢...,speech,speech fed listens helping youth thrive discus...,0.017246
805,20220321,speech Chair Pro Tempore Jerome H. Powell At ...,speech,speech chair pro tempore jerome h powell polic...,-0.024148


In [307]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.7,5,150,0.001,6)

torch.Size([452, 1])
IC -0.0118 IC spear -0.03479 P_value 0.46063 loss 0.22933
Epoch: 0, loss: 0.00230
torch.Size([452, 1])
IC -0.01198 IC spear -0.03543 P_value 0.45238 loss 0.24015
Epoch: 1, loss: 0.00169
torch.Size([452, 1])
IC -0.01161 IC spear -0.03609 P_value 0.44409 loss 0.23586
Epoch: 2, loss: 0.00189
torch.Size([452, 1])
IC -0.01104 IC spear -0.03523 P_value 0.45501 loss 0.22592
Epoch: 3, loss: 0.00163
torch.Size([452, 1])
IC -0.01043 IC spear -0.03604 P_value 0.44468 loss 0.21628
Epoch: 4, loss: 0.00132
torch.Size([452, 1])
IC -0.00989 IC spear -0.03622 P_value 0.44233 loss 0.21089
Epoch: 5, loss: 0.00124
torch.Size([452, 1])
IC -0.00946 IC spear -0.03561 P_value 0.45013 loss 0.21083
Epoch: 6, loss: 0.00126
torch.Size([452, 1])
IC -0.0091 IC spear -0.03426 P_value 0.46752 loss 0.21484
Epoch: 7, loss: 0.00117
torch.Size([452, 1])
IC -0.00874 IC spear -0.03322 P_value 0.48106 loss 0.22081
Epoch: 8, loss: 0.00101
torch.Size([452, 1])
IC -0.00831 IC spear -0.03058 P_value 0.51661

In [318]:
def taining(data,ratio,seq_length,features,l2,seed):
  train_X,train_Y,test_X,test_Y=split_data(data,ratio,features,seq_length)
  num_epochs = 400
  learning_rate = 0.01
  input_size = features
  hidden_size = 2
  num_layers = 1
  num_classes = 1
  setup_seed(seed)
  lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=l2)
  from sklearn.metrics import mean_squared_error
  for epoch in range(num_epochs):
      outputs = lstm(train_X)
      optimizer.zero_grad()
      loss = criterion(outputs, train_Y[0])
      loss.backward()
      optimizer.step()
      predTrain = lstm(test_X)
      print(predTrain.shape)
      tmp=pd.DataFrame(list(test_Y))
      tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
      r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
      print('IC',round(tmp.corr().iloc[0,1],5),
            'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
            'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
            'loss',round(r,5))
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
taining(new_raw,0.7,5,150,0.001,3)


torch.Size([453, 1])
IC 0.0196 IC spear -0.01447 P_value 0.75872 loss 0.07931
Epoch: 0, loss: 0.00212
torch.Size([453, 1])
IC 0.02113 IC spear -0.0125 P_value 0.79073 loss 0.07036
Epoch: 1, loss: 0.00105
torch.Size([453, 1])
IC 0.02223 IC spear -0.00905 P_value 0.84769 loss 0.06431
Epoch: 2, loss: 0.00073
torch.Size([453, 1])
IC 0.02289 IC spear -0.00711 P_value 0.8801 loss 0.06121
Epoch: 3, loss: 0.00079
torch.Size([453, 1])
IC 0.02303 IC spear -0.00731 P_value 0.87674 loss 0.0605
Epoch: 4, loss: 0.00088
torch.Size([453, 1])
IC 0.02273 IC spear -0.00703 P_value 0.88143 loss 0.06146
Epoch: 5, loss: 0.00086
torch.Size([453, 1])
IC 0.02208 IC spear -0.00782 P_value 0.86814 loss 0.0635
Epoch: 6, loss: 0.00072
torch.Size([453, 1])
IC 0.02118 IC spear -0.00866 P_value 0.85418 loss 0.06616
Epoch: 7, loss: 0.00055
torch.Size([453, 1])
IC 0.02017 IC spear -0.00944 P_value 0.84126 loss 0.06899
Epoch: 8, loss: 0.00041
torch.Size([453, 1])
IC 0.01917 IC spear -0.01142 P_value 0.80845 loss 0.07156